In [1]:
# init

import math
import numpy as np
import pickle
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
from scipy.misc import imresize
from skimage import feature

In [2]:
# helpers

# plot images, list are dislayed in subplots
def plot(imgs, title=None):
    # make sure input is a list
    if not isinstance(imgs, list):
        imgs = [imgs]
    
    plt.figure()
        
    for i in range(len(imgs)):
        if(title is not None): 
            plt.suptitle(title)
        plt.subplot(1, len(imgs), i+1)
        plt.axis("off")
        plt.imshow(imgs[i])
        

# create empty list of given size
def empty(size):
    return [None]*size

print('done')

done


In [3]:
# choose data_set to be used
data_set = 'train'

# load data
data = pickle.load(open("./data_pp/data-%s.pkl" % data_set, 'rb'))
c = len(data['rgb'])
print('%i samples' % c)

r = range(c)

print('done')

38000 samples
done


In [66]:
for i in range(50, 100):
    if(not data['valid'][i]):
        continue 
        
    img = data['dep'][i]
    img[img==0] = 255
    
    d = 127
    while((img<d).sum() > 500):
        d -= 1
    
    orientations=4
    pixels_per_cell=(20,20)
    cells_per_block=(4,4)
    
    hog_features, hog_image = feature.hog(img, orientations, pixels_per_cell, cells_per_block, visualise=True)
    
#     plot([img<d, hog_image, data['rgb'][i]])